In [1]:
from openai import OpenAI
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, text

In [2]:
api_key = os.environ.get('openai_api_key')

In [3]:
df = pd.read_csv('sales_data_sample.csv')

In [4]:
# create a temp databese in memory
temp_db = create_engine('sqlite:///:memory:', echo=True)


In [5]:
#push the pandas df ---> to temp db for us to be able to interact
data = df.to_sql(name='Sales', con=temp_db)

2024-01-24 15:56:44,039 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-24 15:56:44,043 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2024-01-24 15:56:44,043 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-24 15:56:44,044 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2024-01-24 15:56:44,044 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-01-24 15:56:44,048 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSIZE" TEXT
)


20

In [6]:
#SQL query on TEMP DB
with temp_db.connect() as conn:
    result = conn.execute(text(
        "SELECT SUM(SALES) FROM Sales GROUP BY QTR_ID LIMIT 5"
        ))

2024-01-24 15:56:44,830 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-24 15:56:44,831 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales GROUP BY QTR_ID LIMIT 5
2024-01-24 15:56:44,832 INFO sqlalchemy.engine.Engine [generated in 0.00156s] ()
2024-01-24 15:56:44,833 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
result.all()

[(2350817.7300000004,),
 (2048120.2999999986,),
 (1758910.8099999994,),
 (3874780.01,)]

In [8]:
#describe the actual database you are using to gpt; say this is the table, these are the columns
def create_table_definition(df, table_name):
    prompt = f"""### sqlite SQL table, with it properties:
    #
    # {table_name}({','.join(str(col) for col in df.columns)})
    #
    """
    return prompt


In [9]:
print(create_table_definition(df, table_name='Sales'))

### sqlite SQL table, with it properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    


In [10]:
def prompt_input():
    nlp_text = input("What you want from the table? ")
    return nlp_text

In [11]:
def combine_prompts(df,table_name, query_prompt):
    table_definition = create_table_definition(df, table_name)
    query_init_string = f'### A query to answer: {query_prompt}\nSELECT'
    return table_definition+query_init_string

In [12]:
nlp_text = prompt_input() #NLP
combine_prompts(df=df, table_name='Sales', query_prompt=nlp_text) # DF + query that does... + NLP


'### sqlite SQL table, with it properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    #\n    ### A query to answer: a\nSELECT'

In [14]:
client = OpenAI(api_key=api_key)



In [18]:
response = client.completions.create(
    model="davinci-002",
    prompt=combine_prompts(df=df, table_name='Sales', query_prompt=nlp_text),
    max_tokens=100,
    temperature=0,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["###", ';']
)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}